<a href="https://colab.research.google.com/github/GregUOC/Privacy-Precerving-ML/blob/main/Heart_Disease_CKKS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opendatasets --upgrade --quiet
!pip install tenseal

In [ ]:
#Author : Grigorios Pachis
import opendatasets as od
# Assign the Kaggle data set URL into variable
dataset = 'https://www.kaggle.com/datasets/naveengowda16/logistic-regression-heart-disease-prediction/data'
# Using opendatasets let's download the data sets
od.download(dataset)

Skipping, found downloaded files in "./logistic-regression-heart-disease-prediction" (use force=True to force download)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df=pd.read_csv('/content/logistic-regression-heart-disease-prediction/framingham_heart_disease.csv')

In [ ]:
df.head(20)

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0
5,0,43,2.0,0,0.0,0.0,0,1,0,228.0,180.0,110.0,30.30,77.0,99.0,0
6,0,63,1.0,0,0.0,0.0,0,0,0,205.0,138.0,71.0,33.11,60.0,85.0,1
7,0,45,2.0,1,20.0,0.0,0,0,0,313.0,100.0,71.0,21.68,79.0,78.0,0
8,1,52,1.0,0,0.0,0.0,0,1,0,260.0,141.5,89.0,26.36,76.0,79.0,0
9,1,43,1.0,1,30.0,0.0,0,1,0,225.0,162.0,107.0,23.61,93.0,88.0,0


In [ ]:
count_final_column1 = (df.iloc[:, -1] == 1).sum()
print("Count of samples with 1 in the final column:", count_final_column1)

count_final_column = (df.iloc[:, -1] == 0).sum()
print("Count of samples with 0 in the final column:", count_final_column)


Count of samples with 1 in the final column: 644
Count of samples with 0 in the final column: 3594


In [ ]:
nan_check = df.isnull()
print(nan_check)

       male    age  education  currentSmoker  cigsPerDay  BPMeds  \
0     False  False      False          False       False   False   
1     False  False      False          False       False   False   
2     False  False      False          False       False   False   
3     False  False      False          False       False   False   
4     False  False      False          False       False   False   
...     ...    ...        ...            ...         ...     ...   
4233  False  False      False          False       False   False   
4234  False  False      False          False       False   False   
4235  False  False      False          False       False    True   
4236  False  False      False          False       False   False   
4237  False  False      False          False       False   False   

      prevalentStroke  prevalentHyp  diabetes  totChol  sysBP  diaBP    BMI  \
0               False         False     False    False  False  False  False   
1               False    

In [ ]:
df.fillna(df.mean(), inplace=True)
#df['glucose'].fillna(df['glucose'].mean(), inplace=True) for individual collumns

In [ ]:
nan_check = df.isnull() #checking if there are any nan values. False=!Nan True=Nan
print(nan_check)

       male    age  education  currentSmoker  cigsPerDay  BPMeds  \
0     False  False      False          False       False   False   
1     False  False      False          False       False   False   
2     False  False      False          False       False   False   
3     False  False      False          False       False   False   
4     False  False      False          False       False   False   
...     ...    ...        ...            ...         ...     ...   
4233  False  False      False          False       False   False   
4234  False  False      False          False       False   False   
4235  False  False      False          False       False   False   
4236  False  False      False          False       False   False   
4237  False  False      False          False       False   False   

      prevalentStroke  prevalentHyp  diabetes  totChol  sysBP  diaBP    BMI  \
0               False         False     False    False  False  False  False   
1               False    

In [ ]:
pip install imbalanced-learn

In [ ]:
from imblearn.over_sampling import SMOTE

X = df.drop(columns='TenYearCHD')
y = df['TenYearCHD']
smote = SMOTE(sampling_strategy='auto', random_state=42)  # auto will balance all classes
X_resampled, y_resampled = smote.fit_resample(X, y)

In [ ]:
df_resampled = pd.DataFrame(X_resampled)
df_resampled['TenYearCHD'] = y_resampled

In [ ]:
df_resampled.head(40)

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.00000,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.000000,0
1,0,46,2.00000,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.000000,0
2,1,48,1.00000,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.000000,0
3,0,61,3.00000,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.000000,1
4,0,46,3.00000,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.000000,0
5,0,43,2.00000,0,0.0,0.0,0,1,0,228.0,180.0,110.0,30.30,77.0,99.000000,0
6,0,63,1.00000,0,0.0,0.0,0,0,0,205.0,138.0,71.0,33.11,60.0,85.000000,1
7,0,45,2.00000,1,20.0,0.0,0,0,0,313.0,100.0,71.0,21.68,79.0,78.000000,0
8,1,52,1.00000,0,0.0,0.0,0,1,0,260.0,141.5,89.0,26.36,76.0,79.000000,0
9,1,43,1.00000,1,30.0,0.0,0,1,0,225.0,162.0,107.0,23.61,93.0,88.000000,0


In [ ]:
count_final_column1 = (df_resampled.iloc[:, -1] == 1).sum()
print("Count of samples with 1 in the final column:", count_final_column1)

count_final_column = (df_resampled.iloc[:, -1] == 0).sum()
print("Count of samples with 0 in the final column:", count_final_column)

Count of samples with 1 in the final column: 3594
Count of samples with 0 in the final column: 3594


Data preprocessing has been completed.I have created a balanced dataset  so that there are as many Positives as negatives, so that the model wont be as biased.


In [ ]:
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [ ]:
df= df_resampled.drop(['education'], axis = 1)
data=df
data.head()

,male,age,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [ ]:
cols_to_be_normalized = ["age","cigsPerDay","totChol","sysBP","diaBP","BMI","heartRate","glucose"]
cols_not_to_be_normalized = ["male","currentSmoker","BPMeds","prevalentStroke","prevalentHyp","diabetes","TenYearCHD"]

normalized = normalize(data[cols_to_be_normalized])
boolean = data[cols_not_to_be_normalized]
df_normalized = pd.DataFrame(normalized,columns = cols_to_be_normalized)
df_boolean = pd.DataFrame(boolean,columns=cols_not_to_be_normalized)
# df_final = pd.concat([df_normalized,df_boolean],axis = 1)
# df_final = df_normalized.join(df_boolean)
df_final = df_normalized.merge(df_boolean,left_index=True, right_index=True)

df_final.head(20)

,age,cigsPerDay,totChol,sysBP,diaBP,BMI,heartRate,glucose,male,currentSmoker,BPMeds,prevalentStroke,prevalentHyp,diabetes,TenYearCHD
0,0.148753,0.000000,0.743764,0.404302,0.266992,0.102868,0.305134,0.293691,1,0,0.0,0,0,0,0
1,0.144425,0.000000,0.784921,0.379902,0.254314,0.090203,0.298270,0.238616,0,0,0.0,0,0,0,0
2,0.154476,0.064365,0.788469,0.410326,0.257459,0.081550,0.241368,0.225277,1,1,0.0,0,0,0,0
3,0.190605,0.093740,0.703050,0.468700,0.296843,0.089303,0.203103,0.321841,0,1,0.0,0,1,0,1
4,0.131263,0.065631,0.813259,0.370960,0.239697,0.065917,0.242551,0.242551,0,1,0.0,0,0,0,0
5,0.126811,0.000000,0.672391,0.530835,0.324399,0.089357,0.227079,0.291959,0,0,0.0,0,1,0,0
6,0.220005,0.000000,0.715890,0.481916,0.247942,0.115625,0.209529,0.296832,0,0,0.0,0,0,0,1
7,0.125666,0.055852,0.874077,0.279258,0.198273,0.060543,0.220614,0.217821,0,1,0.0,0,0,0,0
8,0.156107,0.000000,0.780534,0.424791,0.267183,0.079134,0.228156,0.237162,1,0,0.0,0,1,0,0
9,0.130834,0.091280,0.684598,0.492911,0.325564,0.071837,0.282967,0.267754,1,1,0.0,0,1,0,0


In [ ]:
X = df_final.drop("TenYearCHD",axis = 1)
Y = df_final["TenYearCHD"]
X = np.array(X)
Y = np.array(Y)
X_train,X_test,y_train,y_test = train_test_split(X,Y,random_state = 1,test_size = 0.2)


This is all with data processing. Our data  now are normalized,and split into train,test sets.

In [ ]:
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
print ("Accuracy : ", accuracy_score(y_test, y_pred))
print (classification_report(y_test, y_pred))

Accuracy :  0.6216968011126565
              precision    recall  f1-score   support

           0       0.61      0.69      0.65       721
           1       0.64      0.55      0.59       717

    accuracy                           0.62      1438
   macro avg       0.62      0.62      0.62      1438
weighted avg       0.62      0.62      0.62      1438



In [ ]:
# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train= torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)  # Ensuring shape is (3391, 1)
X_test= torch.tensor(X_test, dtype=torch.float32)
y_test= torch.tensor(y_test, dtype=torch.float32).reshape(-1, 1)

print(f"X_train has shape: {X_train.shape}")
print(f"y_train has shape: {y_train.shape}")
print(f"X_test has shape: {X_test.shape}")
print(f"y_test has shape: {y_test.shape}")

print(f"Min value in y_train: {y_train.min()}")
print(f"Max value in y_train: {y_train.max()}")

X_train has shape: torch.Size([5750, 14])
y_train has shape: torch.Size([5750, 1])
X_test has shape: torch.Size([1438, 14])
y_test has shape: torch.Size([1438, 1])
Min value in y_train: 0.0
Max value in y_train: 1.0


In [ ]:
import random
torch.random.manual_seed(73)
random.seed(73)

def clean_dataset(df):

    # We could either populate the missing values or just
    # remove the entire column if not needed any more.

    #df["education"] = df["education"].fillna(df["education"].mean())
    df["cigsPerDay"] = df["cigsPerDay"].fillna(df["cigsPerDay"].mean())
    df["BPMeds"] = df["BPMeds"].fillna(df["BPMeds"].mean())
    df["totChol"] = df["totChol"].fillna(df["totChol"].mean())
    df["BMI"] = df["BMI"].fillna(df["BMI"].mean())
    df["heartRate"] = df["heartRate"].fillna(df["heartRate"].mean())
    df["glucose"] = df["glucose"].fillna(df["glucose"].mean())

    # remove rows with missing values
    df = df.dropna()

    new_df = df.groupby("TenYearCHD")
    df = new_df.apply(lambda x: x.sample(new_df.size().min(), random_state=73).reset_index(drop=True))
    return df
# Split the dataset into train(80%) and test(20%)
def split_train_and_test(X, y, ratio=0.2):
    idxs = [i for i in range(len(X))]
    random.shuffle(idxs)
    # delimiter between test and train data
    delim = int(len(X) * ratio)
    test_idxs, train_idxs = idxs[:delim], idxs[delim:]
    return X[train_idxs], y[train_idxs], X[test_idxs], y[test_idxs]


def standardize_data():
    global df
    clean_dataset(df)
    df = (df - df.mean()) / df.std()
    X = torch.tensor(df.values).float()
    y = torch.tensor(df["TenYearCHD"].values).float().unsqueeze(1)
    df = df.drop("TenYearCHD",axis = 1)
    return split_train_and_test(X, y)


def randomize_data(m=1024, n=2):
    # data separable by the line `y = x`
    X_train = torch.randn(m, n)
    X_test = torch.randn(m // 2, n)
    y_train = (X_train[:, 0] >= X_train[:, 1]).float().unsqueeze(0).t()
    y_test = (X_test[:, 0] >= X_test[:, 1]).float().unsqueeze(0).t()
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = standardize_data()

y_train = (y_train - y_train.min()) / (y_train.max() - y_train.min())
y_test = (y_test - y_test.min()) / (y_test.max() - y_test.min())

In [ ]:
class LogisticRegression(torch.nn.Module):

    def __init__(self, n_features):
        super(LogisticRegression, self).__init__()
        self.lr = torch.nn.Linear(n_features, 1)

    def forward(self, x):
        output = torch.sigmoid(self.lr(x))

        return output

class LogisticRegression2(torch.nn.Module):

    def __init__(self, n_features):
        super(LogisticRegression2, self).__init__()
        #self.lr = torch.nn.Linear(n_features, n_features)
        #self.dropout = nn.Dropout(p=0)
        self.lr2 = torch.nn.Linear(n_features, 1)

    def forward(self, x):
        #x = torch.sigmoid(self.lr(x))
        #x = self.dropout(x)
        output = torch.sigmoid(self.lr2(x))

        return output

n_features = X_train.shape[1]
model = LogisticRegression(n_features)

# Setting up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Move model to the device
#model.to(device)
# Gradient descent with learning rate of 0.001
optim = torch.optim.SGD(model.parameters(), lr=1) #0.01 0.71 0.001 0.69
bceloss = torch.nn.BCELoss()
epochs=20
def train_model(model, optim, bceloss, X, y, epochs):
    for ep in range(1, epochs+1):
        optim.zero_grad()
        result = model(X)

        loss = bceloss(result, y)
        loss.backward()
        optim.step()
        print(f"Loss at epoch number {ep}: {loss.data}")
    return model

model = train_model(model, optim, bceloss, X_train, y_train, epochs)

Loss at epoch number 1: 0.787005603313446
Loss at epoch number 2: 0.46608734130859375
Loss at epoch number 3: 0.33741670846939087
Loss at epoch number 4: 0.26170483231544495
Loss at epoch number 5: 0.21236540377140045
Loss at epoch number 6: 0.17806218564510345
Loss at epoch number 7: 0.1529923677444458
Loss at epoch number 8: 0.1339389979839325
Loss at epoch number 9: 0.11900123208761215
Loss at epoch number 10: 0.10699310898780823
Loss at epoch number 11: 0.09713996946811676
Loss at epoch number 12: 0.08891578018665314
Loss at epoch number 13: 0.08195158839225769
Loss at epoch number 14: 0.0759812593460083
Loss at epoch number 15: 0.07080818712711334
Loss at epoch number 16: 0.06628409773111343
Loss at epoch number 17: 0.06229513883590698
Loss at epoch number 18: 0.05875241011381149
Loss at epoch number 19: 0.05558564513921738
Loss at epoch number 20: 0.05273845046758652


In [ ]:
def accuracy(model, X, y):
    out = model(X)
    print(out)
    correct = torch.abs(y - out) < 0.5

    return correct.float().mean()
p_accuracy = accuracy(model, X_test, y_test)
print(f"Accuracy on plain test_set using a Logistic Regression with a single neural network with a single node. : {p_accuracy}")

tensor([[0.9384],
        [0.0504],
        [0.0407],
        ...,
        [0.0511],
        [0.9818],
        [0.9452]], grad_fn=<SigmoidBackward0>)
Accuracy on plain test_set using a Logistic Regression with a single neural network with a single node. : 1.0


In [ ]:
def accuracy(model, X, y):
    out = model(X)
    correct2=0
    for i in range(len(y)):
      if out[i]<0.5:
        out[i]=0
      else :
        out[i]=1
      #print("y: ",y[i],"out :",out[i])
      if out[i]==y[i]:
        correct2+=1

    total=len(y)
    print("correct :",correct2)
    print("total :",total)
    return correct2/total
p_accuracy = accuracy(model, X_test, y_test)
print(f"Accuracy on plain test_set using a Logistic Regression with a single neural network with a single node. : {p_accuracy}")

correct : 1437
total : 1437
Accuracy on plain test_set using a Logistic Regression with a single neural network with a single node. : 1.0


Now we are going to work with encrypted data


In [ ]:
import tenseal as ts

In [ ]:

class EncryptedLR:

    def __init__(self, torch_lr):
        # TenSEAL processes lists and not torch tensors,
        # so we take out the parameters from the PyTorch model
        self.weight = torch_lr.lr.weight.data.tolist()[0]
        self.bias = torch_lr.lr.bias.data.tolist()

    def forward(self, enc_x):
        # We don't need to perform sigmoid as this model
        # will only be used for evaluation, and the label
        # can be deduced without applying sigmoid
        enc_out = enc_x.dot(self.weight) + self.bias
        #enc_out = enc_out.rescale()
        return enc_out

    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)

    def encrypt(self, context):
        self.weight = ts.ckks_vector(context, self.weight)
        self.bias = ts.ckks_vector(context, self.bias)

    def decrypt(self, context):
        self.weight = self.weight.decrypt()
        self.bias = self.bias.decrypt()

eelr = EncryptedLR(model)

In [ ]:
'''
# Define parameters
poly_mod_degree =4096
coeff_mod_bit_sizes = [40, 20,  40]
'''
# Specify parameters
poly_mod_degree = 8192
coeff_mod_bit_sizes = [40, 21, 21, 21, 21, 21, 21, 40] #this is much more taxing but provides a scale that will allow us to do the nessesary computations

# Create TenSEALContext
ctx_eval = ts.context(ts.SCHEME_TYPE.CKKS, poly_mod_degree, -1, coeff_mod_bit_sizes)

# Scale of ciphertext to use
ctx_eval.global_scale = 2 ** 40
# This key is needed for doing dot-product operations
ctx_eval.generate_galois_keys()

In [ ]:
from time import time
t_start = time()
enc_X_test = [ts.ckks_vector(ctx_eval, x.tolist()) for x in X_test]
t_end = time()
print(f"Encryption of the test-set took {int(t_end - t_start)} second(s)")

Encryption of the test-set took 15 second(s)


In [ ]:
# encrypt the model's parameters
eelr.encrypt(ctx_eval)



In [ ]:
def encrypted_evaluation(model, enc_X_test, y_test):
    t_start = time()

    correct = 0
    for enc_X, y in zip(enc_X_test, y_test):
        # encrypted evaluation
        #enc_X = ts.ckks_vector(context, X, scale=2**40)
        enc_out = model(enc_X) #scale out of bounds shows up here potential solutions include , decrease the size of X_test, rescale enc_X_test,change encryption parameters
        # plain comparison
        out = enc_out.decrypt()
        out = torch.tensor(out)
        out = torch.sigmoid(out)
        if torch.abs(out - y) < 0.5:
            correct += 1

    t_end = time()
    print(f"Evaluated test_set of {len(X_test)} entries in {int(t_end - t_start)} seconds")
    print(f"Accuracy: {correct}/{len(X_test)} = {correct / len(X_test)}")
    return correct / len(X_test)

#investigate why this is running out of bounds
encrypted_accuracy = encrypted_evaluation(eelr, enc_X_test, y_test)
diff_accuracy = p_accuracy - encrypted_accuracy
print(f"Difference between plain and encrypted accuracies: {diff_accuracy}")
if diff_accuracy < 0:
    print("we got a better accuracy on the encrypted test-set! The noise was on our end ")

Evaluated test_set of 1437 entries in 141 seconds
Accuracy: 1437/1437 = 1.0
Difference between plain and encrypted accuracies: 0.0


Moving on to Encrypted training

In [ ]:
class EncryptedLR:

    def __init__(self, torch_lr):
        self.weight = torch_lr.lr.weight.data.tolist()[0]
        self.bias = torch_lr.lr.bias.data.tolist()
        # we accumulate gradients and counts the number of iterations
        self._delta_w = 0
        self._delta_b = 0
        self._count = 0

    def forward(self, enc_x):
        enc_out = enc_x.dot(self.weight) + self.bias
        enc_out = EncryptedLR.sigmoid(enc_out)
        return enc_out

    def backward(self, enc_x, enc_out, enc_y):
        out_minus_y = (enc_out - enc_y)
        self._delta_w += enc_x * out_minus_y
        self._delta_b += out_minus_y
        self._count += 1

    def update_parameters(self):
        if self._count == 0:
            raise RuntimeError("You should at least run one forward iteration")
        # update weights
        # We use a small regularization term to keep the output
        # of the linear layer in the range of the sigmoid approximation
        self.weight -= self._delta_w * (1 / self._count) + self.weight * 0.05
        self.bias -= self._delta_b * (1 / self._count)
        # reset gradient accumulators and iterations count
        self._delta_w = 0
        self._delta_b = 0
        self._count = 0

    @staticmethod
    def sigmoid(enc_x):
        # We use the polynomial approximation of degree 3
        # sigmoid(x) = 0.5 + 0.197 * x - 0.004 * x^3
        # from https://eprint.iacr.org/2018/462.pdf
        # which fits the function pretty well in the range [-5,5]
        return enc_x.polyval([0.5, 0.197, 0, -0.004])

    def plain_accuracy(self, x_test, y_test):
        # evaluate accuracy of the model on
        # the plain (x_test, y_test) dataset
        w = torch.tensor(self.weight)
        b = torch.tensor(self.bias)
        out = torch.sigmoid(x_test.matmul(w) + b).reshape(-1, 1)
        correct = torch.abs(y_test - out) < 0.5
        return correct.float().mean()

    def encrypt(self, context):
        self.weight = ts.ckks_vector(context, self.weight)
        self.bias = ts.ckks_vector(context, self.bias)

    def decrypt(self):
        self.weight = self.weight.decrypt()
        self.bias = self.bias.decrypt()

    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)
# Specify parameters
poly_mod_degree = 8192
coeff_mod_bit_sizes = [40, 21, 21, 21, 21, 21, 21, 40]
# create TenSEALContext
ctx_training = ts.context(ts.SCHEME_TYPE.CKKS, poly_mod_degree, -1, coeff_mod_bit_sizes)
ctx_training.global_scale = 2 ** 21
ctx_training.generate_galois_keys()
t_start = time()
enc_X_train = [ts.ckks_vector(ctx_training, x.tolist()) for x in X_train]
enc_y_train = [ts.ckks_vector(ctx_training, y.tolist()) for y in y_train]
t_end = time()
print(f"Encryption of the training_set took {int(t_end - t_start)} seconds")

In [ ]:
eelr = EncryptedLR(LogisticRegression(n_features))
accuracy = eelr.plain_accuracy(X_test, y_test)
print(f"Accuracy at epoch 0 is {accuracy}")
epochs=5
#eelr.to(device)
times = []
for epoch in range(epochs):
    eelr.encrypt(ctx_training)

    # if you want to keep an eye on the distribution to make sure
    # the function approxiamation is still working fine
    # WARNING: this operation is time consuming
    # encrypted_out_distribution(eelr, enc_X_train)

    t_start = time()
    for enc_X, enc_y in zip(enc_X_train, enc_y_train):
        enc_out = eelr.forward(enc_X)
        eelr.backward(enc_X, enc_out, enc_y)
    eelr.update_parameters()
    t_end = time()
    times.append(t_end - t_start)

    eelr.decrypt()
    accuracy = eelr.plain_accuracy(X_test, y_test)
    print(f"Accuracy at epoch #{epoch + 1} is {accuracy}")


print(f"\nAverage time per epoch: {int(sum(times) / len(times))} seconds")
print(f"Final accuracy is {accuracy}")

diff_accuracy = p_accuracy - accuracy
print(f"Difference between plain and encrypted accuracies: {diff_accuracy}")
if diff_accuracy < 0:
    print("We got a better accuracy when training on encrypted data!")